In [1]:
from underthesea import sent_tokenize, word_tokenize
import torch
from transformers import AutoModelForTokenClassification, AutoTokenizer

In [2]:
tokenizer = AutoTokenizer.from_pretrained('vinai/phobert-base', use_fast=False)
phobert_ner = AutoModelForTokenClassification.from_pretrained('/opt/github/Phobert-Named-Entity-Reconigtion/phobert-ner-mck')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
label_list = [
    "B-CNAME",
    "B-HNAME",
    "I-CNAME",
    "I-HNAME",
    "MCK",
    "O"
]

In [4]:
content = "Công ty Cổ phần Dầu Thực vật Tường An (mã CK: TAC) đã công bố BCTC quý 1/2022. Cụ thể, doanh thu thuần đạt 1.697 tỷ đồng tăng 7,2% so với cùng kỳ. Tuy nhiên giá vốn hàng bán chiếm tới 94% trong doanh thu khiến lãi gộp chỉ còn hơn 98 tỷ đồng, giảm 55% so với quý 1/2021. Trong kỳ chi phí bán hàng giảm mạnh từ 113,5 tỷ đồng xuống còn hơn 21 tỷ đồng, chi phí QLDN cũng thấp hơn cùng kỳ. Do lãi gộp thấp nên kết quả TAC vẫn báo lãi sau thuế giảm 32% so với cùng kỳ, đạt 53 tỷ đồng – tương đương EPS đạt 1.559 đồng."

In [6]:
paragraphs = [p for p in content.split("\n") if p != '']

tokenize_sentences = []
for paragraph in paragraphs:
    sentences = sent_tokenize(paragraph)
    for sentence in sentences:
        if sentence != '':
            tokenize_sentence = word_tokenize(sentence, format="text")
            tokenize_sentences.append(tokenize_sentence)

for sentence in tokenize_sentences:
    list_ids = tokenizer(sentence)['input_ids']
    # if sentence is longer than 256 tokens, ignore token 257 onward
    # if len(list_ids) >= 256:
        # list_ids = list_ids[0:255]
    input_ids = torch.tensor([list_ids])

    tokens = tokenizer.convert_ids_to_tokens(list_ids)
    outputs = phobert_ner(input_ids).logits
    predictions = torch.argmax(outputs, dim=2)
    for i in [(token, label_list[prediction]) for token, prediction in zip(tokens, predictions[0].numpy())]:
        print(i)
        if i[1] != 'O':
            print(i)
    print('---------------------------------------------')

('Công_ty', 'B-CNAME')
('Cổ_phần', 'I-CNAME')
('Dầu', 'I-CNAME')
('Thực_vật', 'I-CNAME')
('Tường_@@', 'I-CNAME')
('An', 'I-CNAME')
('TAC', 'MCK')
---------------------------------------------
---------------------------------------------
---------------------------------------------
---------------------------------------------
('TAC', 'MCK')
---------------------------------------------
